In [1]:
import os
import os.path as osp
import sys
sys.path.append(osp.abspath('..'))

import numpy as np
import torch
from tqdm import tqdm_notebook

import config
from datasets.gtzan import GTZAN_SPEC
from dbn import DBN

%load_ext autoreload
%autoreload 2

In [2]:
# Random seeds
np.random.seed(1234)
MIN_SEGMENTS = 2

DBN_CHECKPOINT = 'dbn_checkpoints/checkpoint_layer_0_epoch_100.pt'

In [3]:
train_set = GTZAN_SPEC(phase='val', test_size=0.1, val_size=0.4,
                       min_segments=MIN_SEGMENTS, randomized=True)
test_set = GTZAN_SPEC(phase='test', test_size=0.1, val_size=0.4, min_segments=MIN_SEGMENTS)

print('Train:', len(train_set))
print('Test:', len(test_set))
print('Shape:', train_set[0][0].shape)

Train: 400
Test: 100
Shape: (2, 221, 301)


In [4]:
Ewhiten = None
with open('pca_whiten_mat.npy', 'rb') as f:
    Ewhiten = np.load(f)
assert Ewhiten is not None

for x_ in train_set[0][0]:
    print(Ewhiten.dot(x_).shape)

(80, 301)
(80, 301)


In [9]:
# aggregate over all frames, calculate simple summary statistics
# such as average, max or standard deviation for each channel
x_train = np.empty((400 * MIN_SEGMENTS, 512))
y_train = np.empty((400 * MIN_SEGMENTS,))
x_test = np.empty((100 * MIN_SEGMENTS, 512))
y_test = np.empty((100 * MIN_SEGMENTS,))
dbn = None
with open(DBN_CHECKPOINT, 'rb') as f:
    dbn = torch.load(f)


def populate_features(src_dataset, dst_x, dst_y):
    for sample_idx, (x, y) in tqdm_notebook(enumerate(src_dataset), total=len(src_dataset)):
        for segment_idx, x_ in enumerate(x):
            x_ = Ewhiten.dot(x_)
            x_ = torch.from_numpy(x_)
            x_ = x_.type(torch.FloatTensor)
            x_ = x_.view(-1)[None, ...]
            x_ = x_.to(0)
            print(x_)
            x_ = dbn.rbms[0].v2h(x_)[1].squeeze_().cpu().numpy()
            print(x_)
            raise Exception()
            idx = sample_idx * MIN_SEGMENTS + segment_idx
            dst_x[idx] = x_
            dst_y[idx] = y

populate_features(train_set, x_train, y_train)
populate_features(test_set, x_test, y_test)

tensor([[ 1.0854e+00,  9.8969e-01,  1.3881e+00,  ..., -8.0051e-03,
         -2.5946e-02, -9.9991e-03]], device='cuda:0')
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1.

Exception: 

In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

params = {
    'C': [10**x for x in range(-3, 3)],
    'gamma': [10**x for x in range(-8, 0)],
    'random_state': [1234],
}

rs = GridSearchCV(SVC(), params, n_jobs=-1, cv=10, verbose=1)
rs.fit(x_train, y_train)
print(rs.best_score_)
print(rs.best_params_)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 160 tasks      | elapsed:   10.2s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   27.9s finished


0.19875
{'C': 0.001, 'gamma': 1e-08, 'random_state': 1234}


In [7]:
from sklearn.metrics import accuracy_score

svm = SVC(**rs.best_params_)
svm.fit(x_train, y_train)

print('Train score:', accuracy_score(y_train, svm.predict(x_train)))
print('Test score:', accuracy_score(y_test, svm.predict(x_test)))

Train score: 0.19875
Test score: 0.2
